# Advent of Code 2021

> I mean, if 10 years from now, when you are doing something quick and dirty, you suddenly visualize that I am looking over your shoulders and say to yourself "Dijkstra would not have liked this," well, that would be enough immortality for me.

-- Edsger W. Dijkstra

## Imports and definitions

In [144]:
#type: ignore
from math import *
from collections import Counter
from itertools import tee, repeat, count, islice, permutations, product
from functools import reduce
from statistics import median, mean
from dataclasses import dataclass
from operator import mul, add, or_
from typing import List, Set
from collections.abc import Generator
from abc import abstractmethod, ABCMeta
import numpy as np
import pandas as pd
import copy


def prod(u): 
    """
    Older Pythons don't have this.
    """
    return reduce(mul, u)


def inputfunc(day, kind='lines', testing=False):
    """
    Generator to read input files.
    """
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'raw':
            text = open(filename)

        def inner():
            return func(f=text)
        return inner
    return gen


def convolution(M, K, T=np.int64):
    """Return the convolution product M * K"""
    mx, my = M.shape
    kx, ky = K.shape
    px = mx + 2 * (kx - 1)
    py = my + 2 * (ky - 1)
    P = np.zeros((px, py), T)
    P[kx-1:mx+kx-1, ky-1:my+ky-1] = M
    O = np.zeros((mx + kx - 1, my + ky - 1), T)

    for y in range(my + ky - 1):
        for x in range(mx + kx - 1):
            O[x, y] = (K * P[x:x+kx, y:y+ky]).sum()

    return O


## [Day 1 - Sonar Sweep](https://adventofcode.com/2021/day/1)

In [ ]:
@inputfunc(1)
def input_1(*, f):
    return [int(x) for x in f]


def count_increasing_slides(a, n):
    return sum(1 for x, y in zip(a, a[n:]) if x < y)


A = count_increasing_slides(input_1(), 1)
assert A == 1624

A = count_increasing_slides(input_1(), 3)
assert A == 1653

## [Day 2 - Dive!](https://adventofcode.com/2021/day/2)

In [ ]:
@inputfunc(2)
def input_2(*, f):
    return [(x, int(y)) for x, y in [x.split() for x in f]]


def pilot_1(l):
    x, y = 0, 0
    for c, n in l:
        if c == 'forward':
            x += n
        elif c == 'down':
            y += n
        elif c == 'up':
            y -= n
    return x * y


def pilot_2(l):
    x, y, aim = 0, 0, 0
    for c, n in l:
        if c == 'forward':
            x += n
            y += aim * n
        elif c == 'down':
            aim += n
        elif c == 'up':
            aim -= n
    return x * y


A = pilot_1(input_2())
assert A == 1654760

A = pilot_2(input_2())
assert A == 1956047400

## [Day 3 - Binary Diagnostic](https://adventofcode.com/2021/day/3)

This uses the naive algorithm, it's a literal transcription of the actual problem description. A far superior approach would be to sort the array of numbers, which can be done in $O(n)$ in a myriad of ways (for example repeatedly applying a stable variant of counting-sort). At that point, lookups for which segment to keep at each iteration are
trivial.

However, because of the problem conditions it just won't matter, both run in milliseconds anyway.

In [ ]:
@inputfunc(3)
def input_3(*, f):
    return f


def power_consumption(l):
    N = len(l)
    W = len(l[0])
    ones_count = [0] * W
    for s in l:
        for i, c in enumerate(reversed(s)):
            if c == '1':
                ones_count[i] += 1
    gamma = sum(2 ** e for e, b in enumerate(ones_count) if b > N/2)
    epsilon = (2 ** W - 1) - gamma
    return gamma * epsilon


def life_support_rating(l):
    def bisect_on(l, predicate):
        out_true, out_false = tee((u, predicate(u)) for u in l)
        return [u for u, p in out_true if p], [u for u, p in out_false if not p]

    def oxygen_generator_rating(l):
        index = 0
        while len(l) > 1:
            zeroes, ones = bisect_on(l, lambda u: u[index] == '0')
            l = zeroes if len(zeroes) > len(ones) else ones
            index += 1
        return int(next(iter(l), None), 2)

    def co2_scrubber_rating(l):
        index = 0
        while len(l) > 1:
            zeroes, ones = bisect_on(l, lambda u: u[index] == '0')
            l = ones if len(ones) < len(zeroes) else zeroes
            index += 1
        return int(next(iter(l), None), 2)

    return oxygen_generator_rating(l) * co2_scrubber_rating(l)


A = power_consumption(input_3())
assert A == 3320834

A = life_support_rating(input_3())
assert A == 4481199

## [Day 4 - Giant Squid](https://adventofcode.com/2021/day/4)

In [ ]:
class BingoCard:
    def __init__(self, m):
        self.matrix = m
        self.num_rows = len(m)
        self.num_cols = len(m[0])
        self.hit_by_row = [0] * self.num_rows
        self.hit_by_col = [0] * self.num_cols
        self.reverse_map = {}
        for nr, r in enumerate(m):
            for nc, num in enumerate(r):
                self.reverse_map[num] = (nr, nc)

    def hit_num(self, n):
        if n not in self.reverse_map:
            return False
        nr, nc = self.reverse_map.pop(n)
        self.hit_by_row[nr] += 1
        if self.hit_by_row[nr] == self.num_cols:
            return True
        self.hit_by_col[nc] += 1
        if self.hit_by_col[nc] == self.num_rows:
            return True
        return False

    def sum_remaining(self):
        return sum(x for x in self.reverse_map)


@inputfunc(4, kind='chunks')
def input_4(*, f):
    seq = [int(x) for x in f[0].split(',')]
    cards = []
    for c in f[1:]:
        lines = c.split('\n')
        card = BingoCard([[int(u) for u in v.split()] for v in lines])
        cards.append(card)

    return cards, seq


def play_bingo_1(cards, seq):
    for num in seq:
        for card in cards:
            if card.hit_num(num):
                return num * card.sum_remaining()


def play_bingo_2(cards, seq):
    alive = set(cards)
    for num in seq:
        hits = set()
        for card in alive:
            if card.hit_num(num):
                if len(alive) == 1:
                    return num * card.sum_remaining()
                hits.add(card)
        alive -= hits


A = play_bingo_1(*input_4())
assert A == 33348

A = play_bingo_2(*input_4())
assert A == 8112

## [Day 5 - Hydrotermal Venture](https://adventofcode.com/2021/day/5)


Disappoitingly, the naive algorithm. I have thought about it for a while and short of reimplementing from scratch a full-blown R Tree-backed database table, I am completely clueless as to how you would solve this.

In [ ]:
class Board:
    def __init__(self):
        self._point_count = Counter()

    def put(self, x1, y1, x2, y2, with_diagonals=False):
        def get_range_for(c1, c2):
            if c1 == c2:
                return repeat(c1)
            elif c1 > c2:
                return range(c1, c2 - 1, -1)
            else:
                return range(c1, c2 + 1, 1)

        range_x = get_range_for(x1, x2)
        range_y = get_range_for(y1, y2)

        if not with_diagonals and x1 != x2 and y1 != y2:
            return

        for x, y in zip(range_x, range_y):
            self._point_count.update({(x, y): 1})

    def overlaps(self):
        return set(p for p, n in self._point_count.items() if n >= 2)


@inputfunc(5)
def input_5(*, f):
    return [
        tuple(
            int(a)
            for a in line.replace('->', ',').split(',')
        )
        for line in f
    ]


def find_overlaps(l, with_diagonals=False):
    board = Board()
    for t in l:
        board.put(*t, with_diagonals=with_diagonals)
    return len(board.overlaps())


A = find_overlaps(input_5())
assert A == 5092

A = find_overlaps(input_5(), with_diagonals=True)
assert A == 20484

## [Day 6 - Lanternfish](https://adventofcode.com/2021/day/6)


Automorphisms of finite-dimensional real vector spaces are what plebeians call matrices.

In [ ]:
@inputfunc(6, kind='single')
def input_6(*, f):
    return Counter(int(u) for u in f.split(','))


def get_lanternfish(state, itn):
    T = np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0]
    ])

    A = np.array([state.get(i, 0) for i in range(9)])
    return np.sum(np.linalg.matrix_power(T, itn) @ A)


A = get_lanternfish(input_6(), 80)
assert A == 359344

A = get_lanternfish(input_6(), 256)
assert A == 1629570219571

## [Day 7 - The Treachery of Whales](https://adventofcode.com/2021/day/7)

### Proof


Let $f_k(x)$ be the cost for the $k$-th crab to meet at $x$. Then the total cost $c(x) = \sum_k f_k(x)$ is minimized only if:

$$ 
    \frac{\partial}{\partial x} \sum_k f_k(x) = 0
$$

#### Case 1

Let the initial position of the $k$-th crab be $p_k$, and let $N$ be the number of crabs. Then $f_k(x) = \left| x - p_k\right|$. The derivative of the total cost function is then:

$$
    \frac{\partial}{\partial x} \sum_k \left| x - p_k \right| =
    \sum_k \sigma(x - p_k)
$$

Where $\sigma$ is the sign function, that is, $\sigma(0) = 0$ and $\sigma(x) = x / |x|$ for all $x \neq 0$. It trivially follows that $c^{\prime}(x) = 0$ if and only if $x$ is a median of $\{f_k\}_k$. Before mathematicians get a fucking aneurism from this: yes, of course $c$ isn't continuous, but it's continuous everywhere except on a finite subset of $\mathbb{R}$. It's trivial to check everything still works on those isolated points.

#### Case 2

Using the same notation as case 1, $f_k(x) = 1/2 \left( \left| x - p_k\right| + \left( x - p_k \right)^{2} \right)$. Therefore:

$$
    \frac{\partial c(x)}{\partial x} = 0 \Leftrightarrow 
    \left( \sum_k {\sigma(x - p_k) + 2x - 2p_k} \right) = 0 \Leftrightarrow
    \left( \sum_k {\sigma(x - p_k)} \right) + 2Nx - 2 \sum_k p_k = 0
$$

Shifting around the terms we get:

$$
    \frac{\partial c(x)}{\partial x} = 0 \Leftrightarrow
    x = \frac{1}{N} \sum_k p_k  - \frac{1}{2N} \sum_k \sigma(x - f_k)
$$

Because $\left| \sigma(u) \right| \leq 1$,

$$
    \frac{1}{2N} \sum_k \sigma(x - f_k) \le \frac{1}{2}
$$

Therefore:

$$
    \left\lfloor -\frac{1}{2} + {\frac{1}{N} \sum_k p_k} \right\rfloor
    \le x \le
    \left\lceil \frac{1}{2} + {\frac{1}{N} \sum_k p_k} \right\rceil
$$

### Code

In [ ]:
@inputfunc(7, kind='single')
def input_7(*, f):
    return [int(u) for u in f.split(',')]


def cost_linear(i):
    m = int(median(i))
    return sum(abs(x - m) for x in i)


def cost_quadratic(i):
    def cost_for_crab(x, m):
        d = abs(x - m)
        return d * (d + 1) // 2

    # The mean is not guaranteed to be integral.
    # We simply "try" both the lower integer part and the
    # higher integer part and keep the best.
    al, ar = floor(mean(i)), ceil(mean(i))
    return min(
        sum(cost_for_crab(x, al) for x in i),
        sum(cost_for_crab(x, ar) for x in i)
    )


A = cost_linear(input_7())
assert A == 344605

A = cost_quadratic(input_7())
assert A == 93699985

## [Day 8 - Seven Segment Search](https://adventofcode.com/2021/day/8)

In [ ]:
@dataclass
class DisplayInstance:
    patterns: List[Set[str]]
    outputs: List[Set[str]]

    def get_output_value(self):
        patterns_bylen = {
            n: [set(x) for x in self.patterns if len(x) == n]
            for n in {2, 3, 4, 5, 6, 7}
        }

        x = [None] * 10

        # The pattern with 2 segments must be a one.
        x[1] = next(iter(patterns_bylen[2]))

        # The pattern with 3 segments must be a seven.
        x[7] = next(iter(patterns_bylen[3]))

        # The pattern with 4 segments must be a four.
        x[4] = next(iter(patterns_bylen[4]))

        # The pattern with 7 segments must be an eight.
        x[8] = next(iter(patterns_bylen[7]))

        # The patterns with 6 segments bust represent
        # zero, six and nine.
        for p in patterns_bylen[6]:
            # Nine is the only one that "contains" a four.
            if p > x[4]:
                x[9] = p
            # If it's not a nine, it's either a zero or a six.
            # Zero "contains" a one, and six doesn't.
            elif p > x[1]:
                x[0] = p
            else:
                x[6] = p

        # The remaining patterns with 5 segments must
        # represent two, three and five.
        for p in patterns_bylen[5]:
            # Three is the only one that "contains" a one.
            if p > x[1]:
                x[3] = p
            # If it's not a three, it's either a five or a two.
            # Six "contains" a five, but not a two.
            elif x[6] > p:
                x[5] = p
            else:
                x[2] = p

        return sum(
            n * 10**e
            for e, p in enumerate(reversed(self.outputs))
            for n, i in enumerate(x)
            if i == p
        )


@inputfunc(8)
def input_8(*, f):
    return [
        DisplayInstance(*[
            [set(u) for u in x.strip().split()]
            for x in l.split('|')
        ])
        for l in f
    ]


A = sum(sum(1 for u in x.outputs if len(u) in {2, 3, 4, 7}) for x in input_8())
assert A == 521

A = sum(x.get_output_value() for x in input_8())
assert A == 1016804

## [Day 9 - Smoke Basin](https://adventofcode.com/2021/day/9)

In [ ]:
class HeightMap:
    def __init__(self, matrix):
        self._matrix = matrix
        self._width = len(matrix)
        self._height = len(matrix[0])

    def point(self, x, y):
        return self._matrix[x][y]

    def get_neighbors_of(self, x, y):
        return [
            (x+dx, y+dy, self.point(x+dx, y+dy))
            for dx, dy in {(1, 0), (-1, 0), (0, 1), (0, -1)}
            if 0 <= x+dx < self._width and 0 <= y+dy < self._height
        ]

    def get_basin_of(self, x, y):
        seen = {(x, y)}
        q = [(x, y, self.point(x, y))]
        while len(q) != 0:
            x, y, p = q.pop()
            for x1, y1, p1 in self.get_neighbors_of(x, y):
                if (x1, y1) not in seen and p1 > p and p1 != 9:
                    q.append((x1, y1, p1))
                    seen.add((x1, y1))
        return seen

    def get_lowpoints(self):
        return [
            (x, y, self.point(x, y))
            for x in range(self._width)
            for y in range(self._height)
            if all(self.point(x, y) < v for _, _, v in self.get_neighbors_of(x, y))
        ]


@inputfunc(9)
def input_9(*, f):
    return [[int(c) for c in x] for x in f]


def sum_lowpoints(i):
    hm = HeightMap(i)
    return sum(1 + p for _, _, p in hm.get_lowpoints())


def largest_basins(i, n):
    hm = HeightMap(i)
    basins_sizes = [len(hm.get_basin_of(x, y))
                    for x, y, _ in hm.get_lowpoints()]
    return prod(sorted(basins_sizes, reverse=True)[:n])



A = sum_lowpoints(input_9())
assert A == 456

A = largest_basins(input_9(), 3)
assert A == 1047744

## [Day 10 - Syntax Scoring](https://adventofcode.com/2021/day/10)

In [ ]:
@inputfunc(10)
def input_10(*, f):
    return f


@dataclass
class UnmatchedChar:
    char: str

    def score(self):
        scores = {
            ')': 3,
            ']': 57,
            '}': 1197,
            '>': 25137
        }
        return scores[self.char]


@dataclass
class IncompleteLine:
    char: list[str]

    def score(self):
        scr = str.maketrans('([{<', '1234')
        return int(''.join(self.char).translate(scr)[::-1], 5)


def check_line(i):
    chars = {
        '(': ')',
        '[': ']',
        '{': '}',
        '<': '>'
    }

    def is_opening(c):
        return c in chars.keys()

    s = []
    for c in i:
        if is_opening(c):
            s.append(c)
        else:
            if c != chars[s.pop()]:
                return UnmatchedChar(c)

    return IncompleteLine(s)

def syntax_all(i):
    res = [check_line(l) for l in i]
    return sum(u.score() for u in res if isinstance(u, UnmatchedChar))

def autocomplete_all(i):
    res = [check_line(l) for l in i]
    return median(u.score() for u in res if isinstance(u, IncompleteLine))


A = syntax_all(input_10())
assert A == 319329

A = autocomplete_all(input_10())
assert A == 3515583998

## [Day 11 - Dumbo Octopus](https://adventofcode.com/2021/day/11)

In [ ]:
@inputfunc(11)
def input_11(*, f):
    return np.array([[int(u) for u in s.strip()] for s in f])
    

def time_step(i, T=np.int64):
    K = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], T)
    count_fired = 0
    elegible = np.ones(i.shape, bool)
    i += np.ones(i.shape, T)
    fired = np.zeros(i.shape, T) + np.ones(i.shape, T) * (i > 9) * elegible
    while fired.sum() > 0:
        count_fired += fired.sum()
        elegible &= (fired <= 0)
        i += convolution(fired, K)[1:-1, 1:-1]
        fired = np.zeros(i.shape, T) + np.ones(i.shape, T) * (i > 9) * elegible

    return i * np.ones(i.shape, T) * (i < 10), count_fired

def count_fired_over_ts(i, n):
    count_fired = 0
    for _ in range(n):
        i, n = time_step(i)
        count_fired += n
    return count_fired



def find_first_sync(i):
    for j in count(1):
        i, _ = time_step(i)
        if i.sum() == 0:
            return j


A = count_fired_over_ts(input_11(), 100)
assert A == 1661

A = find_first_sync(input_11())
assert A == 334

## [Day 12 - Passage Pathing](https://adventofcode.com/2021/day/12)

In [ ]:
@dataclass
class Node:
    label: str
    kind: str
    adj: list


@inputfunc(12)
def input_12(*, f):
    nodes = {
        'start': Node('start', 'start', []),
        'end': Node('end', 'end', [])
    }

    def node_kind(s):
        if s == 'start' or s == 'end':
            return s
        elif s.isupper():
            return 'big'
        else:
            return 'small'

    for l in f:
        u, v = l.split('-')
        un = nodes.get(u, Node(u, node_kind(u), []))
        vn = nodes.get(v, Node(v, node_kind(v), []))
        un.adj.append(v)
        vn.adj.append(u)
        nodes[u] = un
        nodes[v] = vn

    return nodes


def breadth_search_1(graph):
    small = {}
    for n, k in (
        enumerate(k for k, v in graph.items() if v.kind == 'small')
    ):
        small[k] = 2 ** n

    def visited(state):
        return {k for k, v in small.items() if state & v}

    Q = [(0, 'start')]
    N = {(0, 'start'): 1}
    acc = 0

    while len(Q) != 0:
        state, nodeid = Q.pop()
        n = N[(state, nodeid)]
        del N[(state, nodeid)]
        for v in graph[nodeid].adj:
            if v == 'start':
                continue
            if v == 'end':
                acc += n
                continue
            if v in visited(state):
                continue
            nstate = state | small.get(v, 0)
            if (nstate, v) in N:
                N[(nstate, v)] += n
            else:
                Q.append((nstate, v))
                N[(nstate, v)] = n

    return acc


def breadth_search_2(graph):
    small = {}
    for n, k in (
        enumerate(k for k, v in graph.items() if v.kind == 'small')
    ):
        small[k] = 2 ** n

    def visited(state):
        return {k for k, v in small.items() if state & v}

    Q = [(0, False, 'start')]
    N = {(0, False, 'start'): 1}
    acc = 0

    while len(Q) != 0:
        state, double, nodeid = Q.pop()
        n = N[(state, double, nodeid)]
        del N[(state, double, nodeid)]
        for v in graph[nodeid].adj:
            if v == 'start':
                continue
            if v == 'end':
                acc += n
                continue
            if v in visited(state):
                if double:
                    continue
                else:
                    ndouble = True
            else:
                ndouble = double
            nstate = state | small.get(v, 0)
            if (nstate, ndouble, v) in N:
                N[(nstate, ndouble, v)] += n
            else:
                Q.append((nstate, ndouble, v))
                N[(nstate, ndouble, v)] = n

    return acc


A = breadth_search_1(input_12())
assert A == 3495

A = breadth_search_2(input_12())
assert A == 94849

## [Day 13 - Transparent Origami](https://adventofcode.com/2021/day/13)

In [ ]:
@inputfunc(13)
def input_13(*, f):
    dots, folds = [], []
    for s in f:
        if s.startswith('fold'):
            _, _, d, n = s.replace('=', ' ').split(' ')
            folds.append((d, int(n)))
        elif s != '':
            y, x = s.split(',')
            dots.append((int(x), int(y)))
    sx, sy = 1 + max(x for x, _ in dots), 1 + max(y for _, y in dots)
    M = np.zeros((sx+1 - sx % 2, sy+1 - sy % 2), bool)
    for p in dots:
        M[p] = True

    return M, folds


def fold_along(M, s, n):
    if s == 'y':
        G, H = M[:n, :], np.flipud(M[n+1:, ])
    elif s == 'x':
        G, H = M[:, :n], np.fliplr(M[:, n+1:])
    return G | H


def fold_along_all(M, l):
    return reduce(lambda X, f: fold_along(X, *f), l, M)

def print_matr(S):
    sx, sy = S.shape
    s = ""
    for x in range(sx):
        for y in range(sy):
            s += "#" if S[x, y] else " "
        s += '\n'
    return s


M, folds = input_13()
A = fold_along(M, *folds[0]).sum()
assert A == 666

print(print_matr(fold_along_all(M, folds)))

This is absolutely metal, as always, advent of code doesn't disappoint!

## [Day 14 - Extended Polymerization](https://adventofcode.com/2021/day/14)


Yet again, linear algebra. Pairs of consecutive letters are the dimesions of the vector space, the number of occurences of each pair at a given iteration is a vector, and the transformation map defines a linear map over vectors. Let $m$ be the size of the alphabet and let $n$ be the number of iterations. There are two solutions with an interesting tradeoff:

* Represent the initial vector $v$ as an array, and manually apply $n$ times the morphism. This has a time complexity of $\Theta(m^2n)$.

* Represent the linear map as a $m^2 \times m^2$ matrix, compute $M^n$, then multiply it by the initial vector. This has a time complexity of $\Theta(m^4 \log{n})$.

Both solutions have a space complexity of $\Theta(n^2)$. I'll do the second because it's more interesting. I also believe it's much more efficient in practice as it can fully take advantage of vectorization intrinsics and is generally very tight: just plain math, no hashtable lookups.

The actual solution is one line, everything else is just shuffling around data.

In [ ]:
@inputfunc(14)
def input_14(*, f):
    m = {}
    init, l_start, l_end = None, None, None
    for l in f:
        if not init:
            init = [*zip(l, l[1:])]
            l_start, l_end = l[0], l[-1]
            continue
        if l == '':
            continue
        (u1, u2), v = l.replace('->', ' ').split()
        m[(u1, u2)] = ((u1, v), (v, u2))

    num = {e: n for n, e in enumerate(m)}

    M = np.zeros((len(num), len(num)), np.int64)
    V = np.zeros(len(num), np.int64)

    for u, (v1, v2) in m.items():
        M[num[v1], num[u]] += 1
        M[num[v2], num[u]] += 1

    for u in init:
        V[num[u]] += 1

    return M, V, l_start, l_end, {n: e for e, n in num.items()}


def polymer_count_after(steps, M, V, l_start, l_end, d):
    F = np.linalg.matrix_power(M, steps) @ V
    C = Counter({l_start: 1, l_end: 1})
    for i, n in enumerate(F):
        v1, v2 = d[i]
        C.update({v1: n})
        C.update({v2: n})

    return Counter({c: u//2 for c, u in C.items()})


C = polymer_count_after(10, *input_14())
A = max(C.values()) - min(C.values())
assert A == 3906

C = polymer_count_after(40, *input_14())
A = max(C.values()) - min(C.values())
assert A == 4441317262452

## [Day 15 - Chiton](https://adventofcode.com/2021/day/15)

In [ ]:
class WrappingHeightMap:
    def __init__(self, matrix, wrap_max=1):
        self._matrix = np.array(matrix, np.uint32)
        mx, my = self._matrix.shape
        self._wrap_matrix = np.empty((mx * wrap_max, my * wrap_max), np.uint32)
        for i in range(wrap_max):
            for j in range(wrap_max):
                self._wrap_matrix[i*mx:(i+1)*mx, j*my:(j+1)
                                  * my] = (self._matrix + i + j - 1) % 9 + 1

    def shape(self):
        return self._matrix.shape

    def wrap_shape(self):
        return self._wrap_matrix.shape

    def get_neighbors_of(self, x, y):
        ww, wh = self._wrap_matrix.shape
        n = []
        if x != 0:
            n.append((x-1, y, self._wrap_matrix[x-1, y]))
        if x != ww - 1:
            n.append((x+1, y, self._wrap_matrix[x+1, y]))
        if y != 0:
            n.append((x, y-1, self._wrap_matrix[x, y-1]))
        if y != wh - 1:
            n.append((x, y+1, self._wrap_matrix[x, y+1]))
        return n


class PriorityQueue:
    def __init__(self):
        self._H = []
        self._map = {}
        self._len = 0

    def __len__(self):
        return self._len

    def _parent(self, n):
        if n == 0:
            return None
        else:
            w = n + 1
            return w // 2 - 1

    def _children(self, n):
        if self._len > 2 * n + 2:
            return 2 * n + 1, 2 * n + 2
        elif self._len == 2 * n + 2:
            return 2 * n + 1, None
        else:
            return None, None

    def _swap(self, a, b):
        self._map[self._H[a][1]], self._map[self._H[b][1]] = b, a
        self._H[a], self._H[b] = self._H[b], self._H[a]

    def _heap_decrease(self, n, r):
        self._H[n] = r
        p = self._parent(n)

        while p != None and self._H[n] < self._H[p]:
            self._swap(n, p)
            n, p = p, self._parent(p)

    def __contains__(self, val):
        return val in self._map

    def insert(self, prio, val):
        self._H.append(None)
        self._map[val] = self._len
        self._heap_decrease(self._len, (prio, val))
        self._len += 1

    def peek(self):
        return self._H[0]

    def extract(self):
        top_prio, top_val = self._H[0]
        self._swap(0, self._len - 1)
        del self._H[-1]
        del self._map[top_val]
        self._len -= 1
        c, (l, r) = 0, self._children(0)

        while l or r:
            cprio, _ = self._H[c]
            lprio, _ = self._H[l] if l else (inf, inf)
            rprio, _ = self._H[r] if r else (inf, inf)

            if cprio <= lprio and cprio <= rprio:
                break
            elif lprio < rprio:
                self._swap(c, l)
                c = l
            else:
                self._swap(c, r)
                c = r

            l, r = self._children(c)

        return top_prio, top_val

    def decrease_prio(self, newprio, val):
        idx = self._map[val]
        prio, _ = self._H[idx]
        self._heap_decrease(idx, (newprio, val))


def dijkstra(f, wrap_max):
    m = WrappingHeightMap(f, wrap_max=wrap_max)
    start = (0, 0)
    target = tuple(d - 1 for d in m.wrap_shape())

    M = {(start): 0}
    Q = PriorityQueue()
    Q.insert(0, start)

    while len(Q) != 0:
        v, (x, y) = Q.extract()

        for xn, yn, vn in m.get_neighbors_of(x, y):
            if v + vn >= M.get((xn, yn), inf):
                continue

            M[(xn, yn)] = v + vn

            if (xn, yn) in Q:
                Q.decrease_prio(v + vn, (xn, yn))
            else:
                Q.insert(v + vn, (xn, yn))

        if (x, y) == target:
            return M[(x, y)]

    return inf


@inputfunc(15)
def input_15(*, f):
    return [[int(n) for n in x] for x in f]


A = dijkstra(input_15(), 1)
assert A == 619

A = dijkstra(input_15(), 5)
assert A == 2922

## [Day 16 - Packet Decoder](https://adventofcode.com/2021/day/16)

It wouldn't be Christmas without recursive descent parsers, would it?

In [ ]:
@dataclass
class Packet(metaclass=ABCMeta):
    version: int

    @abstractmethod
    def eval(self):
        ...


@dataclass
class LiteralVal(Packet):
    payload: int

    def eval(self):
        return self.payload


@dataclass
class Operator(Packet, metaclass=ABCMeta):
    subpackets: List[Packet]


class Sum(Operator):
    def eval(self):
        return sum(x.eval() for x in self.subpackets)


class Product(Operator):
    def eval(self):
        return prod(x.eval() for x in self.subpackets)


class Minimum(Operator):
    def eval(self):
        return min(x.eval() for x in self.subpackets)


class Maximum(Operator):
    def eval(self):
        return max(x.eval() for x in self.subpackets)


class Gt(Operator):
    def eval(self):
        fst, snd, *_ = self.subpackets
        return int(fst.eval() > snd.eval())


class Lt(Operator):
    def eval(self):
        fst, snd, *_ = self.subpackets
        return int(fst.eval() < snd.eval())


class Eq(Operator):
    def eval(self):
        fst, snd, *_ = self.subpackets
        return int(fst.eval() == snd.eval())


class bits_gen(Generator):
    def __init__(self, cg):
        self.read = 0
        self._cg = cg
        self._al = []

    def send(self, val):
        return sum(
            2**j for j in range(val-1, -1, -1)
            if self._next_bit()
        )

    def _next_bit(self):
        self.read += 1
        if len(self._al) == 0:
            c = next(self._cg)
            self._al = [
                1 if int(c, 16) & j else 0
                for j in [1, 2, 4, 8]
            ]
        return self._al.pop()

    def throw(self):
        raise StopIteration


def parse_literal_val(g):
    acc = 0
    cont = g.send(1)

    while True:
        acc = 16 * acc + g.send(4)
        if cont:
            cont = g.send(1)
        else:
            break

    return acc


def parse_operator(g):
    subpackets = []
    len_opt = g.send(1)

    if len_opt == 0:
        target_len = g.send(15) + g.read
        while g.read < target_len:
            subpackets.append(parse_packet(g))
    else:
        for _ in range(g.send(11)):
            subpackets.append(parse_packet(g))

    return subpackets


def parse_packet(g):
    ver = g.send(3)
    ptype = g.send(3)

    if ptype == 4:
        return LiteralVal(version=ver, payload=parse_literal_val(g))
    elif ptype == 0:
        return Sum(version=ver, subpackets=parse_operator(g))
    elif ptype == 1:
        return Product(version=ver, subpackets=parse_operator(g))
    elif ptype == 2:
        return Minimum(version=ver, subpackets=parse_operator(g))
    elif ptype == 3:
        return Maximum(version=ver, subpackets=parse_operator(g))
    elif ptype == 5:
        return Gt(version=ver, subpackets=parse_operator(g))
    elif ptype == 6:
        return Lt(version=ver, subpackets=parse_operator(g))
    elif ptype == 7:
        return Eq(version=ver, subpackets=parse_operator(g))


def sum_ver_numbers(p):
    if isinstance(p, LiteralVal):
        return p.version
    elif isinstance(p, Operator):
        return p.version + sum(sum_ver_numbers(x) for x in p.subpackets)


@inputfunc(16, kind='single')
def input_16(*, f):
    return iter(f)


A = sum_ver_numbers(parse_packet(bits_gen(input_16())))
assert A == 979

A = parse_packet(bits_gen(input_16())).eval()
assert A == 277110354175

## [Day 17 - Trick Shot](https://adventofcode.com/2021/day/17)

### Proof


Finding the initial velocities for which the probe eventually steps into the target zone is equivalent to finding all solutions of some diophantine equations. Let the target zone limits be $X_l, X_h$ and $Y_l, Y_h$, for lower and upper bounds of $X$ and $Y$ coordinates respectively.


#### Lemma 1: Coordinates at time $t$

Assume that the initial velocities of the probe are $(s_x, s_y)$. Because they don't interact, we can compute the $x$ and $y$ coordinates $(p_x(t; s_x), p_y(t; s_y))$ of the position $p(t; (s_x, s_y))$ after $t$ steps independently.

After $t$ steps, $p_y$ is simply:

$$
    p_y(t; s_y) = \sum_{k = 0}^{t-1} {s_y - k} = ts_y + \frac{1}{2} (t-1)t
$$

The expression for $p_x(t; s_y)$ is similar but more complicated. First of all, we observe that $p_x(t; 0) = 0$ for all $t$. Similarly, by symmetry, $p_x(t; -s_y) = -p_x(t; s_y)$. We may therefore assume without loss of generality that $s_y > 0$.

If $t < s_x$, the expression is the same as the one for the $y$ coordinate:

$$
    p_x(t; s_x) = \sum_{k = 0}^{t-1} {s_x - k} = ts_x + \frac{1}{2} (t-1)t
$$

After $s_x$ steps, the probe has $x$-velocity 0 in all subsequent steps, therefore for all $t \ge s_x$:

$$
    p_x(t; s_x) = \sum_{k = 1}^{s_x} k = \frac{1}{2} s_x(s_x + 1)
$$

#### Definition: Feisable pairs

A pair $(s_x, t)$ where $s_x$ is an integer and $t$ is a nonnegative integer is feisable for $x$-velocity if and only if $Y_l \le p_y(t; s_y) \le Y_h$.

We define feisable pairs for $y$-velocity in the same way.

#### Lemma 2.a: Feisable pairs for $y$-velocities

Let's now find what initial $y$-velocities are such that the probe eventually intersects the target zone. This is equivalent to finding all pairs $(s_y, t)$, where $s_x$ is an integer and $t$ is a nonnegative integer, such that:

$$
    Y_l \le p_y(t; s_y) \le Y_h
$$

Which we may rewrite, by lemma 1, as:

$$
    Y_l \le ts_y + \frac{1}{2} (t-1)t \le Y_h
$$

By simple manipulation we obtain:

$$
    \frac{2 Y_l + t^2 - t}{2t} \le s_y \le \frac{2 Y_h + t^2 - t}{2t}
$$

#### Lemma 2.b: Upper bound to feisable values of $t$

We can prove the following statement: for no $t \gt 2 \max{\left| Y_l \right|, \left| Y_h \right|}$ there exist integer values of $s_y$ such that:

$$
    \frac{2 Y_l + t^2 - t}{2t} \le s_y \le \frac{2 Y_h + t^2 - t}{2t}
$$

For $s_y$ to satisfy the above inequality, it must hold:

$$
    \left\lceil \frac{Y_l}{t} + \frac{t-1}{2} \right\rceil \le
    \left\lfloor \frac{Y_h}{t} + \frac{t-1}{2} \right\rfloor
$$

Because the fractional part of $(t-1)/2$ is always either $0$ or $1/2$, it must hold:

$$
    \left| \frac{Y_l}{t} \right| \ge \frac{1}{2} \vee \left| \frac{Y_h}{t} \right| \ge \frac{1}{2}
$$

Which is exactly q.e.d.

#### Lemma 3: Feisable pairs for $x$-velocities

Candidate values for initial $x$-velocities may be obtained in the same way as for $y$-velocities when $t < s_x$. There is however an additional case: because the $x$-velocity is eventually zero, for all values of $t$ such that:

$$
    X_l \le \frac{t(t+1)}{2} \le X_h
$$

The pair $(p_x(t; t), t)$ is feisable.

#### Lemma 4: Finding the solutions

Using lemmata 2.a, 2.b and 3, we may obtain a finite number of feisable pairs for $x$- and $y$- velocity. A pair $(s_x, s_y)$ is a solution to the problem if and only if there exists a $t$ such there there exist feisable pairs $(s_x, t)$, $(s_y, t)$.

### Time Complexity

Lemma 4 suggests an algorithm to compute all solutions: generate feisable pairs for both $x$- and $y$- velocity, and then hash join them on the time attribute. While the implementation is straightforward, determining its time complexity is not!

To generate feisable pairs for $y$-velocity, by lemma 2b, we have to try values of $t$ from $1$ to $M = 2\max{\left|Y_l\right|, \left|Y_h\right|}$. At each iteration, we have to scan and insert in a hashtable at most:

$$
    \frac{2 Y_h + t^2 - t}{2t} - \frac{2 Y_l + t^2 - t}{2t}
$$

values. Therefore, the total number of insertions is bounded by:

$$
    \sum_{t=0}^{M} \frac{D}{t} = DH_M
$$

Where $D = Y_h - Y_l$ and $H_M$ is the $M$-th harmonic number. This means that the entire loop takes time at most $O(M + D \log M)$.

The loop that generate feisable pairs for $x$-velocity has the same asymptotic complexity: the only additional operation it performs is a search for the additional case of lemma 3, which takes time at most $O(\sqrt{M})$.

Because hash-joins may be performed in linear time, this means the entire algorithm runs in time bounded by $O(M + D \log M)$.

### Code

In [ ]:
@inputfunc(17, kind='single')
def input_17(*, f):
    _, _, ax, ay = f.replace(',', '').split()
    _, xmin, xmax = ax.replace('..', '=').split('=')
    _, ymin, ymax = ay.replace('..', '=').split('=')
    return int(xmin), int(xmax), int(ymin), int(ymax)


def valid_y(ymin, ymax):
    valid = set()
    for t in range(1, 1 + 2 * max(abs(ymin), abs(ymax))):
        vl = 2 * ymin + t * (t - 1)
        vh = 2 * ymax + t * (t - 1)
        valid |= set(
            ((t, t), x) for x in range(
                ceil(vl / (2 * t)),
                1 + floor(vh / (2 * t))
            )
        )
    return valid


def valid_x(xmin, xmax):
    def helper(xmin, xmax):
        valid = set()
        for t in range(1, max(abs(xmin), abs(xmax))):
            vl = 2 * xmin + t * (t - 1)
            vh = 2 * xmax + t * (t - 1)
            valid |= set(
                ((t, t), x)
                for x in range(
                    ceil(vl / (2 * t)),
                    1 + floor(vh / (2 * t))
                )
                if x > t
            )

        for j in count():
            s = (j * (j+1)) // 2
            if s > xmax:
                break
            elif s >= xmin:
                valid |= {((j, None), j)}

        return valid

    if xmin > xmax:
        return set()
    elif xmin > 0:
        return helper(xmin, xmax)
    elif xmax < 0:
        return {(b, -n) for (b, n) in helper(-xmax, -xmin)}
    elif xmin <= 0 <= xmax:
        pos = helper(0, xmax)
        neg = {(b, -n) for (b, n) in helper(0, -xmin)}
        return pos | neg


def all_solutions(xmin, xmax, ymin, ymax):
    return {
        (w, h)
        for (ml, mh), h in valid_y(ymin, ymax)
        for (rl, rh), w in valid_x(xmin, xmax)
        if mh == rh or (ml >= rl and rh is None)
    }



A = max(h * (h + 1) // 2 for _, h in all_solutions(*input_17()))
assert A == 4560

A = len(all_solutions(*input_17()))
assert A == 3344

## [Day 18 - Snailfish](https://adventofcode.com/2021/day/18)

I refuse to use `eval()` as a matter of principle.

In [ ]:
class SnailfishNum(metaclass=ABCMeta):
    parent = None

    @abstractmethod
    def magnitude(self):
        ...

    @abstractmethod
    def min_subt(self):
        ...

    @abstractmethod
    def max_subt(self):
        ...

    @abstractmethod
    def split(self):
        ...

    @abstractmethod
    def explode(self, n):
        ...

    @classmethod
    def parse_num(cls, s):
        eval_stack = []
        for t in s.replace(',', ' ').replace('[', ' [ ').replace(']', ' ] ').split():
            if t == '[':
                eval_stack.append('[')
            elif t == ']':
                _, left, right = eval_stack[-3:]
                eval_stack = eval_stack[:-3]
                p = Pair(left, right)
                eval_stack.append(p)
            else:
                eval_stack.append(Num(int(t)))
        return eval_stack.pop()

    def __add__(self, other):
        p = Pair(copy.deepcopy(self), copy.deepcopy(other))
        while p.explode(4) or p.split():
            continue
        return p

    def is_left(self):
        if self.parent is None:
            return False
        return self.parent.left == self

    def is_right(self):
        if self.parent is None:
            return False
        return self.parent.right == self

class Num(SnailfishNum):
    def __init__(self, n):
        self.n = n

    def __repr__(self):
        return str(self.n)

    def explode(self, _):
        return False

    def split(self):
        if not self.n >= 10:
            return False
        nl, nr = Num(floor(self.n / 2)), Num(ceil(self.n / 2))
        np = Pair(nl, nr)
        if self.is_left():
            self.parent.left = np
        if self.is_right():
            self.parent.right = np
        return True

    def min_subt(self):
        return self
    
    def max_subt(self):
        return self

    def magnitude(self):
        return self.n


class Pair(SnailfishNum):
    def __init__(self, left, right):
        self.__left = left
        left.parent = self
        self.__right = right
        right.parent = self

    def split(self):
        return self.left.split() or self.right.split()

    def explode(self, k):
        if k > 0:
            return self.left.explode(k-1) or self.right.explode(k-1)
        prev_inorder = self.prev_inorder()
        next_inorder = self.next_inorder()
        if prev_inorder:
            prev_inorder.n += self.left.n
        if next_inorder:
            next_inorder.n += self.right.n
        if self.is_left():
            self.parent.left = Num(0)
        if self.is_right():
            self.parent.right = Num(0)
        return True


    def min_subt(self):
        return self.left.min_subt()

    def max_subt(self):
        return self.right.max_subt()

    def next_inorder(self):
        if self.right:
            self.right.min_subt()
        u, p = self, self.parent
        while p and p.parent and p.left != u:
            u, p = u.parent, p.parent
        if p and p.left == u:
            return p.right.min_subt()
        else:
            return None

    def prev_inorder(self):
        if self.left:
            self.left.max_subt()
        u, p = self, self.parent
        while p and p.parent and p.right != u:
            u, p = u.parent, p.parent
        if p and p.right == u:
            return p.left.max_subt()
        else:
            return None

    def magnitude(self):
        return (
            3 * self.left.magnitude() 
            + 2 * self.right.magnitude()
        )

    def __repr__(self):
        return f"[{repr(self.left)}, {repr(self.right)}]"

    @property
    def left(self):
        return self.__left

    @left.setter
    def left(self, left):
        self.__left = left
        left.parent = self

    @property
    def right(self):
        return self.__right

    @right.setter
    def right(self, right):
        self.__right = right
        right.parent = self


@inputfunc(18)
def input_18(*, f):
    return [SnailfishNum.parse_num(s) for s in f]


A = reduce(add, input_18()).magnitude()
assert A == 4435

A = max((u + v).magnitude() for u in input_18() for v in input_18() if u != v)
assert A == 4802

## [Day 19 - Beacon Scanner](https://adventofcode.com/2021/day/19)

In [169]:
@inputfunc(19, kind='chunks')
def input_19(*, f):
    b = []
    for c in f:
        d = []
        for n, l in enumerate(c.split('\n')):
            if n == 0:
                continue
            d.append(tuple(int(a) for a in l.split(',')))
        b.append(d)
    return b


def can_overlap(soa, sob, mz):
    for p in permute(sob):
        for (ax, ay, az), (bx, by, bz) in product(soa, p):
            ox, oy, oz = ax - bx, ay - by, az - bz
            shifted = {(bbx + ox, bby + oy, bbz + oz) for bbx, bby, bbz in p}
            matching = set(soa) & shifted
            if len(matching) >= mz:
                return shifted, (ox, oy, oz)


def permute(l):
    def permutation_sign(l):
        sign = +1
        for i in range(len(l)):
            for j in range(i, len(l)):
                if l[i] > l[j]:
                    sign = -sign
        return sign
    
    return [
        [(s0 * x[p0], s1 * x[p1], s2 * x[p2]) for x in l]
        for ((p0, p1, p2), (s0, s1, s2)) 
        in product(permutations(range(3)), product((+1, -1), (+1, -1), (+1, -1)))
        if s0 * s1 * s2 == permutation_sign((p0, p1, p2))
    ]


def solve_config(ll):
    coord = {0: set(ll[0])}
    offsets = {0: (0, 0, 0)}

    while len(coord) < len(ll):
        matched, tomatch = set(coord), set(range(len(ll))) - set(coord)
        for prev in matched:
            for curr in tomatch:
                u = can_overlap(coord[prev], set(ll[curr]), 12)
                if u:
                    coord[curr], offsets[curr] = u

    return coord, offsets


def manhattan_distance(a, b):
    ax, ay, az = a
    bx, by, bz = b
    return abs(ax - bx) + abs(ay - by) + abs(az - bz)


B, O = solve_config(input_19())

A = len(reduce(or_, B.values()))
assert A == 465

A = max(manhattan_distance(a, b) for a, b in product(O.values(), O.values()))
assert A == 12149

## [Day 20 - Trench Map](https://adventofcode.com/2021/day/20)

It took me an embarassing amount of time to figure out what was wrong with my initial guess. Out-of-bounds zeros can become ones, they need not remain zeros!

In [20]:
@dataclass
class Picture:
    arr: np.array
    rev: bool

@inputfunc(20, kind='chunks')
def input_20(*, f):
    pattern, img = f
    bit = lambda z: 1 if z == '#' else 0
    pattern = np.array([bit(c) for c in pattern], np.int32)
    img = np.array([[bit(c) for c in r] for r in img.split()], np.int32)
    return pattern, Picture(img, False)


def enhance(pattern, img):
    K = 2 ** np.array([[8, 7, 6], [5, 4, 3], [2, 1, 0]], np.int32)
    MK = np.array([
        pattern[511 - u if img.rev else u] 
        for u in convolution(img.arr, K, T=np.int32)
    ])
    rev = pattern[511] if img.rev else pattern[0]
    img = Picture(1 - MK if rev else MK, rev)
    return img


def enhance_times(pattern, img, n):
    return reduce(lambda u, _: enhance(pattern, u), [*range(n)], img)


xp, xi = input_20()

A = enhance_times(xp, xi, 2).arr.sum()
assert A == 4873

A = enhance_times(xp, xi, 50).arr.sum()
assert A == 16394

## [Day 21 - Dirac Dice](https://adventofcode.com/2021/day/21)

### Proof

Number theory again! As a matter of fact, part one of Day 21 is more interesting than part two, and can be solved entirely with pencil and paper! Yay for $O(0)$ solutions!

It's easy to observe that because the result of dice rolls are deterministic as per the problem description, the $n$-th turn of play will result in dice rolls that add to $6 + 9(n-1)$.
We can therefore determine the square each player lands on after every turn performing arithmetic in the ring $\mathbb{Z} / 10\mathbb{Z}$.

#### Position of player 1 after $k$ turns

Let $s_1$ be the starting position of player 1. Because player 1 takes all odd-numbered turns, his position $p_1(k)$ on the track after the $k$-th of his own turns will be:

$$
    p_1(k) = s_1 + \sum_{r=1}^{k} 6 + 9(2r-2) = s_1 + 6k + 9k(k-1) = s_1 + 6k - k(k-1) = s_1 + k(7-k)
$$

#### Position of player 2 after k turns

Let $s_2$ be the starting position of player 2. Player 2 takes all even-numbered turns, therefore his position $p_2(k)$ after the $k$-th of his own turns will be:

$$
    p_2(k) = s_2 + \sum_{r=1}^{k} 6 + 9(2r - 1) = s_2 + 6k + 9k(k+1) - 9k = s_2 + 7k - k(k+1) = s_2 + k(6-k)
$$

#### Tabulating results

At this point, it's sufficient to exhaustively compute $p_1$ and $p_2$ for all values of $\mathbb{Z}_{10}$. Remember that zeros must be turned into tens, the board described in the problem goes $(1..n)$ rather than the usual $(0..n-1)$.
I'm going to use my own input in the example below, but it can obviously be done for any input in exactly the same way.

In [2]:
s1 = [3 for _ in range(10)] 
s2 = [4 for _ in range(10)]
r1 = [(k * (7 - k)) % 10 for k in range(10)]
r2 = [(k * (6 - k)) % 10 for k in range(10)]
p1 = [(r + s) % 10 or 10 for r, s in zip(s1, r1)]
p2 = [(r + s) % 10 or 10 for r, s in zip(s2, r2)]
pd.DataFrame(data={
    's1': s1, 
    'k(7-k)': r1, 
    'p1 = s1 + k(7-k)': p1, 
    's2': s2, 
    'k(6-k)': r2, 
    'p2 = s2 + k(6-k)': p2
})

,s1,k(7-k),p1 = s1 + k(7-k),s2,k(6-k),p2 = s2 + k(6-k)
0,3,0,3,4,0,4
1,3,6,9,4,5,9
2,3,0,3,4,8,2
3,3,2,5,4,9,3
4,3,2,5,4,8,2
5,3,0,3,4,5,9
6,3,6,9,4,0,4
7,3,0,3,4,3,7
8,3,2,5,4,4,8
9,3,2,5,4,3,7


Adding up all the numbers in the $p_1$ and $p_2$ columns we get $50$ for player 1 and $55$ for player 2. Those are the scores after each player has played ten rounds. This means that after $180$ rounds, player 2 leads $990$ to $900$. The dice has been rolled $180 \times 2 \times 3 = 1080$ times at this point. We can easily simulate the rest of the game: player 2 wins at the end of the $182$-nd round with the score of $1001$ to $912$.

Putting it all together, the dice has been rolled $1092$ times and player 1, who lost, has $912$ points. The answer to the problem is then $1092 \times 912 = \boxed{995904}$.

The code in the section below implements the same idea for arbitrary input.

### Part 2


There are unfortunately no similar ideas (or at least none I could find, of course) for part 2, the nondeterminism makes the state space explode too fast. It's easy enough to solve with dynamic programming, though: if we precompute the probabilities for each possible sum of three rolls, it reduces to counting simple paths on the directed acyclic graph of unique game states. The state space has size at most $\Theta(W^2N^2)$ where $W$ is the winning score and $N$ is the order of the group. For our problem size this means the algorithm terminates nearly instantly even on potato hardware.

### Code

In [42]:
@inputfunc(21)
def input_21(*, f):
    _, _, p1 = f[0].partition(':')
    _, _, p2 = f[1].partition(':')
    return int(p1), int(p2)

def dirac_1(sp1, sp2):
    W = 1000
    cyc1 = [(sp1 + k * (7 - k)) % 10 or 10 for k in range(10)]
    cyc2 = [(sp2 + k * (6 - k)) % 10 or 10 for k in range(10)]
    nrounds = W // max(sum(cyc1), sum(cyc2))
    score1, score2 = nrounds * sum(cyc1), nrounds * sum(cyc2)
    for n, (a, b) in islice(enumerate(zip(cyc1, cyc2)), 1, None):
        score1 += a
        if score1 >= W:
            return (60 * nrounds + 6 * n - 3) * score2
        score2 += b
        if score2 >= W:
            return (60 * nrounds + 6 * n) * score1


def dirac_2(sp1, sp2):
    W = 21

    ROLLS = Counter([a + b + c
        for a in (1, 2, 3)
        for b in (1, 2, 3)
        for c in (1, 2, 3)
    ])

    init = (0, 0, sp1, sp2, 0)
    state_pool = {init: 1}
    active_set = {init}
    winning = [0, 0]

    while len(active_set) > 0:
        pre = active_set.pop()
        cnt = state_pool[pre]
        del state_pool[pre]
        for r, v in ROLLS.items():
            score_up, score_other, pos_up, pos_other, up = pre
            pos_up = ((pos_up + r) % 10) or 10
            score_up += pos_up
            if score_up >= W:
                winning[up] += v * cnt
                continue
            post = (score_other, score_up, pos_other, pos_up, 1-up)
            active_set.add(post)
            state_pool[post] = state_pool.get(post, 0) + v * cnt

    return max(winning)


A = dirac_1(*input_21())
assert A == 995904

A = dirac_2(*input_21())
assert A == 193753136998081

## [Day 22 - Reactor Reboot](https://adventofcode.com/2021/day/22)

The correct way to solve this problem is to use a multidimensional [priority R-tree](https://www.win.tue.nl/~mdberg/Papers/prtree.pdf) to store objects: it's the optimal data structure that supports intersection queries. However, actually implemlementing _that_ would drive me more thoroughly nuts than I already am (have you read section 2.1.3? Come on!). It's unfortunately just too much work.

In [15]:
class Cube:
    def __init__(self, v):
        self.v = v

    def volume(self):
        return prod(h - l + 1 for l, h in self.v)

    def __and__(self, other):
        s = []
        for (tl, th), (vl, vh) in zip(self.v, other.v):
            if th < vl or vh < tl:
                return None
            s.append((max(tl, vl), min(th, vh)))

        return Cube(tuple(s))

    def __hash__(self):
        return hash(self.v)


@inputfunc(22, testing=False)
def input_22(*, f):
    l = []
    tr = str.maketrans('xyz=.,', '      ')
    for x in f:
        w, xl, xh, yl, yh, zl, zh = x.translate(tr).split()
        l.append((w, Cube((
            (int(xl), int(xh)),
            (int(yl), int(yh)),
            (int(zl), int(zh))
        ))))
    return l


def reboot():
    counts = Counter()
    for w, c in input_22():
        new_counts = Counter()
        for oc in counts:
            o = c & oc
            if o is not None:
                new_counts.update({o: -counts[oc]})

        if w == 'on':
            new_counts[c] += 1

        counts.update(new_counts)

    return counts


def sum_volumes(counts, limit):
    def intersect_vol(c):
        if limit:
            c = c & limit
        return c.volume() if c else 0

    return sum(intersect_vol(c) * counts[c] for c in counts)


A = sum_volumes(reboot(), Cube(((-50, 50), (-50, 50), (-50, 50))))
assert A == 610196

A = sum_volumes(reboot(), None)
assert A == 1282401587270826

## [Day 23 - Amphipod](https://adventofcode.com/2021/day/22)

In [ ]:
class Burrow:
    def __init__(self, place):
        self.place = place

    def __hash__(self):
        return hash((
            self.place['room_a'],
            self.place['room_b'],
            self.place['room_c'],
            self.place['room_d'],
            self.place['left_rep'],
            self.place['right_rep'],
            self.place['hallway_left'],
            self.place['hallway_center'],
            self.place['hallway_right']
        ))

    def path_blocked(self, s, e):
        pos_codes = {
            ('room_a', 'room_b'): ['hallway_left'],
            ('room_a', 'room_c'): ['hallway_left', 'hallway_center'],
            ('room_a', 'room_d'): ['hallway_left', 'hallway_center', 'hallway_right'],
            ('room_a', 'rep_left'): [],
            ('room_a', 'rep_right'): ['hallway_left', 'hallway_center', 'hallway_right'],
            ('room_a', 'hallway_left'): [],
            ('room_a', 'hallway_center'): ['hallway_left'],
            ('room_a', 'hallway_right'): ['hallway_left', 'hallway_center'],

            ('room_b', 'room_b'): ['hallway_left'],
            ('room_b', 'room_c'): ['hallway_left', 'hallway_center'],
            ('room_b', 'room_d'): ['hallway_left', 'hallway_center', 'hallway_right'],
            ('room_b', 'rep_left'): [],
            ('room_b', 'rep_right'): ['hallway_left', 'hallway_center', 'hallway_right'],
            ('room_b', 'hallway_left'): [],
            ('room_b', 'hallway_center'): ['hallway_left'],
            ('room_b', 'hallway_right'): ['hallway_left', 'hallway_center'],
        }

    def succ_list(self):
        succ_list = []


## [Day 24 - Arithmetic Logic Unit](https://adventofcode.com/2021/day/24)

This is, yet again, a case where the "correct" solution is unforunately too complex to actually implement.
Because the only operations as per the problem description are additions, multiplications, euclidean divisions and equality tests,
the state of each register may be computed symbolically, i.e. represented by its AST. Inputs are represented by free variables. 
This is very cheap to perform (assuming you have enough memory, which on modern hardware you defintely do). 

At this point, we rebuild the AST of the control register (z) from the bottom up, performing [partition refinement](https://en.wikipedia.org/wiki/Partition_refinement)

In [40]:
def consolidate(state, gb):
    lo = state.groupby(gb).min().reset_index()
    hi = state.groupby(gb).max().reset_index()
    return (
        lo.merge(hi, on=gb)[gb + ['lo_x', 'hi_y']]
        .rename(columns={
            'lo_x': 'lo', 'hi_y': 'hi'
        })
    )

@inputfunc(24)
def input_24(*, f):
    lst = []
    for l in f:
        inst, *args = l.split()
        lst.append((inst, args))
    return lst


def static_recompile(l):
    state = pd.DataFrame(data={
        'w': [0],
        'x': [0],
        'y': [0],
        'z': [0],
        'lo': [0],
        'hi': [0]
    })

    for inst, args in l:
        if inst == 'inp':
            state = state.drop(args[0], axis=1)
            state = consolidate(state, [u for u in ['w', 'x', 'y', 'z'] if u != args[0]])
            df = pd.DataFrame({args[0]: list(range(1, 10))})
            state = state.merge(df, how='cross')
            state['lo'] = 10 * state['lo'] + state[args[0]]
            state['hi'] = 10 * state['hi'] + state[args[0]]
            continue

        try:
            srt = int(args[1])
        except ValueError:
            srt = state[args[1]]

        if inst == 'add':
            state[args[0]] += srt
        elif inst == 'mul':
            state[args[0]] *= srt
        elif inst == 'div':
            state[args[0]] //= srt
        elif inst == 'mod':
            state[args[0]] %= srt
        elif inst == 'eql':
            state[args[0]] = state[args[0]] == srt

    return state


staz = static_recompile(input_24())

A = max(staz[staz['z'] == 0]['hi'])
assert A == 99919765949498

A = min(staz[staz['z'] == 0]['lo'])
assert A == 24913111616151

## [Day 25 - Sea Cucumber](https://adventofcode.com/2021/day/25)

In [2]:
@inputfunc(25)
def input_25(*, f):
    return np.array(
        [[*l] for l in f]
    )

def step_for(A, dr='>'):
    axis = 1 if dr == '>' else 0
    V = np.where(A == dr, 1, 0)
    D = np.where(np.roll(A, 1, axis) == dr, 1, 0) 
    F = np.where(A == '.', 1, 0)
    Dm = D & F
    Vm = np.roll(np.where(Dm == 0, 1, 0), -1, axis)
    N = (Vm & V) | Dm
    return np.where(N == 1, dr, np.where(A == dr, '.', A))

def step(A):
    return step_for(step_for(A, dr='>'), dr='v')

def first_fixpoint(A):
    for j in count(1):
        A, B = step(A), A
        if (A == B).all():
            return j


A = first_fixpoint(input_25())
assert A == 568